In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import geopandas as gpd

aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              2.005219103610955,
              41.48530450682617
            ],
            [
              2.005219103610955,
              41.38238988441918
            ],
            [
              2.170028728650209,
              41.38238988441918
            ],
            [
              2.170028728650209,
              41.48530450682617
            ],
            [
              2.005219103610955,
              41.48530450682617
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

gdf = gpd.GeoDataFrame.from_features(aoi, crs="EPSG:4326")
gdf.total_bounds

array([ 2.0052191 , 41.38238988,  2.17002873, 41.48530451])

TODO: change download to explore, faster and less cost

In [9]:
from spai.data.satellite import utils

locations = utils.get_box_or_gdfs_by_place_name("Madrid")  # returns a list of dicts like this: [ {'name': 'Fraga, Catalunya', 'gdf': <gpd.GeoDataFrame object>}, ...]
locations[0]['gdf']

,geometry
0,"POLYGON ((-3.51792 40.31198, -3.51792 40.64373..."


In [10]:
import ipywidgets
import ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles
from geopandas import GeoSeries

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(41.46183613708533, 2.058000718868186),
    zoom=3
    )

# Add a custom zoom slider
zoom_slider = ipywidgets.IntSlider(description="Zoom level:", min=1, max=10, value=6)
ipywidgets.jslink((zoom_slider, "value"), (m, "zoom"))
widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position="topright")
m.add_control(widget_control1)

selected_geodata = None
def handle_click(event, feature, id, properties):
    global selected_geodata
    selected_geodata = feature
    print(feature)

# Add gdfs to map
geodata_mapping = {}
for idx,location in enumerate(locations):
    geodata = ipyleaflet.GeoData(
        geo_dataframe=location["gdf"],
        style={
            "color": "blue",
            "opacity": 1,
            "weight": 1.9,
            "fillOpacity": 0.5,
        },
        hover_style={"fillColor": "#b08a3e", "fillOpacity": 0.9},

    )
    geodata.on_click(handle_click)
    m.add_layer(geodata)
    # add a marker with location name for each location
    # centroid = location["gdf"]["geometry"].centroid
    # marker = ipyleaflet.Marker(location=(centroid.y[0], centroid.x[0]), title=location["name"] if location["name"] else "Unknown")
    # m.add_layer(marker)

# add a marker with location name for each location
# for location in locations:
#     centroid = location["gdf"]["geometry"].centroid
#     marker = ipyleaflet.Marker(location=(centroid.y[0], centroid.x[0]), title=location["name"] if location["name"] else "Unknown")
#     m.add_layer(marker)

# def handle_interaction(**kwargs):
#     print(kwargs)
#     if kwargs.get('type') == 'click':
#         print(kwargs.get('coordinates'))
#         selected_layer = kwargs.get('coordinates')
#         # check if coordinates are inside any of the gdfs
# m.on_interaction(handle_interaction)
m

Map(center=[41.46183613708533, 2.058000718868186], controls=(ZoomControl(options=['position', 'zoom_in_text', …

{'id': '0', 'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-3.5179163, 40.3119774], [-3.5179163, 40.6437293], [-3.8889539, 40.6437293], [-3.8889539, 40.3119774], [-3.5179163, 40.3119774]]]}}


In [11]:
selected_geodata

{'id': '0',
 'type': 'Feature',
 'properties': {},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-3.5179163, 40.3119774],
    [-3.5179163, 40.6437293],
    [-3.8889539, 40.6437293],
    [-3.8889539, 40.3119774],
    [-3.5179163, 40.3119774]]]}}

### Explore available images

In [12]:
# from spai.data.satellite.utils import get_bbox_from_location_name

# bbox = get_bbox_from_location_name("Barcelona")
# bbox

In [13]:
from spai.data.satellite import explore_satellite_images

# images = explore_satellite_images("Barcelona", ("2023-04-01", "2023-05-01"), cloud_cover=0.2)

bbox = (2.02846789, 41.27036258,  2.27905924, 41.45938995)
images = explore_satellite_images(bbox, ("2023-04-01", "2023-05-01"), cloud_cover=0.2)

images

[{'id': 'S2B_MSIL1C_20230420T104619_N0509_R051_T31TDF_20230420T125145',
  'thumbnail': 'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/T/DF/2023/4/20/0/preview.jpg',
  'date': '2023-04-20T10:59:35Z',
  'cloud_cover': 0.0},
 {'id': 'S2A_MSIL1C_20230412T103621_N0509_R008_T31TDF_20230412T155912',
  'thumbnail': 'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/T/DF/2023/4/12/0/preview.jpg',
  'date': '2023-04-12T10:49:41Z',
  'cloud_cover': 0.0},
 {'id': 'S2A_MSIL1C_20230402T103621_N0509_R008_T31TDF_20230402T160048',
  'thumbnail': 'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/T/DF/2023/4/2/0/preview.jpg',
  'date': '2023-04-02T10:49:40Z',
  'cloud_cover': 0.1}]

### Download image locally

- AOI is a path to GeoJSON

In [ ]:

from spai.storage import Storage
from spai.data.satellite import download_satellite_image

storage = Storage('data')
sensor = 'S2L2A' # or 'S2L1C'
geojson_path = 'aoi.geojson'
path = download_satellite_image(geojson_path,'2023-04-12', sensor, storage)

path

'data/S2L2A_2023-04-12.tif'

- AOI is a GeoJSON

In [ ]:
from spai.storage import Storage
from spai.data.satellite import download_satellite_image

storage = Storage('data')
sensor = 'S2L2A' # or 'S2L1C'
geojson_dict = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "coordinates": [
                    [
                        [2.058000718868186, 41.46183613708533],
                        [2.058000718868186, 41.4318016264732],
                        [2.097234913854237, 41.4318016264732],
                        [2.097234913854237, 41.46183613708533],
                        [2.058000718868186, 41.46183613708533]
                    ]
                ],
                "type": "Polygon"
            }
        }
    ]
}
path = download_satellite_image(geojson_dict,'2023-04-12', sensor, storage)

path

'data/S2L2A_2023-04-12.tif'

- AOI is a GeoDataFrame

In [ ]:
from spai.storage import Storage
from spai.data.satellite import download_satellite_image
import geopandas as gpd

storage = Storage('data')
sensor = 'S2L2A' # or 'S2L1C'
geojson_gdf = gpd.read_file(geojson_path)
path = download_satellite_image(geojson_gdf,'2023-04-12', sensor, storage)

path

'data/S2L2A_2023-04-12.tif'

In [ ]:
from shapely.geometry import box

storage = Storage('data')
sensor = 'S2L2A' # or 'S2L1C'
x = box(2.058000718868186, 41.4318016264732, 2.097234913854237, 41.46183613708533)
path = download_satellite_image(x,'2023-04-12', sensor, storage)

path

'data/S2L2A_2023-04-12.tif'

In [ ]:
gdf = gpd.GeoDataFrame(geometry=[x])
gdf

,geometry
0,"POLYGON ((1.00000 4.00000, 1.00000 3.00000, 2...."


- AOI is a location name

In [ ]:
from spai.storage import Storage
from spai.data.satellite import download_satellite_image

storage = Storage('data')
sensor = 'S2L2A' # or 'S2L1C'
path = download_satellite_image('Madrid','2023-04-12', sensor, storage)

path